# Mavlink Camera
> Mavlink Camera Component for sending commands to a camera on a companion computer or GCS
> The server is on the companion computer and the client is on the ground station PC.

In [ ]:
#| default_exp mavlink.test_camera

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and 
# skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

In [ ]:
#| export
import time
from UAV.mavlink.mavcom import MAVCom, time_since_boot_ms, time_UTC_usec, date_time_str
from UAV.mavlink.component import Component, mavutil, mavlink, MAVLink

from UAV.mavlink.camera_client import *
from UAV.mavlink.camera_server import *
from UAV.utils.display import *
from fastcore.test import *

In [ ]:
#| hide
from fastcore.utils import *
from nbdev.showdoc import *


### Implementation of these commands:
>
> [MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS = 527](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS)
[MAV_CMD_REQUEST_CAMERA_INFORMATION = 523](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_INFORMATION)
[MAV_CMD_REQUEST_CAMERA_SETTINGS = 524](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_SETTINGS)
[MAV_CMD_REQUEST_STORAGE_INFORMATION = 525](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_STORAGE_INFORMATION)
[MAV_CMD_STORAGE_FORMAT = 526](https://mavlink.io/en/messages/common.html#MAV_CMD_STORAGE_FORMAT)
[MAV_CMD_SET_CAMERA_ZOOM = 531](https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_ZOOM)
[MAV_CMD_SET_CAMERA_FOCUS = 532](https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_FOCUS)
[MAV_CMD_IMAGE_START_CAPTURE = 2000](https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_START_CAPTURE)
[MAV_CMD_IMAGE_STOP_CAPTURE = 2001](https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_STOP_CAPTURE)
> 
> [MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION = 2504](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION)
[MAV_CMD_REQUEST_VIDEO_STREAM_STATUS = 2505](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_STATUS)
[MAV_CMD_VIDEO_START_CAPTURE = 2500](https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_START_CAPTURE)
[MAV_CMD_VIDEO_STOP_CAPTURE = 2501](https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_STOP_CAPTURE)
[MAV_CMD_SET_CAMERA_MODE = 530](https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_MODE)
> 
**Note**
The simulated camera is implemented in PX4 [gazebo_camera_manager_plugin.cpp](https://github.com/PX4/PX4-SITL_gazebo-classic/blob/main/src/gazebo_camera_manager_plugin.cpp).


In [ ]:
#| exports
# from pymavlink.dialects.v20 import ardupilotmega as mav
# from pymavlink.dialects.v20.ardupilotmega import MAVLink


NAN = float("nan")

"""
MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS = 527 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS
MAV_CMD_REQUEST_CAMERA_INFORMATION = 521 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_INFORMATION
MAV_CMD_REQUEST_CAMERA_SETTINGS = 522 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_SETTINGS
MAV_CMD_REQUEST_STORAGE_INFORMATION = 525 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_STORAGE_INFORMATION
MAV_CMD_STORAGE_FORMAT = 526 # https://mavlink.io/en/messages/common.html#MAV_CMD_STORAGE_FORMAT
MAV_CMD_SET_CAMERA_ZOOM = 531 # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_ZOOM
MAV_CMD_SET_CAMERA_FOCUS = 532 # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_FOCUS
MAV_CMD_IMAGE_START_CAPTURE = 2000  # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_START_CAPTURE
MAV_CMD_IMAGE_STOP_CAPTURE = 2001  # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_STOP_CAPTURE
MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION = 2504 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION
MAV_CMD_REQUEST_VIDEO_STREAM_STATUS = 2505 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_STATUS
MAV_CMD_VIDEO_START_CAPTURE = 2500 # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_START_CAPTURE
MAV_CMD_VIDEO_STOP_CAPTURE = 2501 # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_STOP_CAPTURE
MAV_CMD_SET_CAMERA_MODE = 530 # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_MODE

"""
CAMERA_INFORMATION = mavlink.MAVLINK_MSG_ID_CAMERA_INFORMATION # https://mavlink.io/en/messages/common.html#CAMERA_INFORMATION
CAMERA_SETTINGS = mavlink.MAVLINK_MSG_ID_CAMERA_SETTINGS # https://mavlink.io/en/messages/common.html#CAMERA_SETTINGS
STORAGE_INFORMATION = mavlink.MAVLINK_MSG_ID_STORAGE_INFORMATION # https://mavlink.io/en/messages/common.html#STORAGE_INFORMATION
CAMERA_CAPTURE_STATUS = mavlink.MAVLINK_MSG_ID_CAMERA_CAPTURE_STATUS # https://mavlink.io/en/messages/common.html#CAMERA_CAPTURE_STATUS
CAMERA_IMAGE_CAPTURED = mavlink.MAVLINK_MSG_ID_CAMERA_IMAGE_CAPTURED # https://mavlink.io/en/messages/common.html#CAMERA_IMAGE_CAPTURED


### Camera Server 
> The server is on the companion computer and is used to receive commands from the camera on the ground station PC.

In [ ]:
show_doc(CameraServer)

---

### CameraServer

>      CameraServer (source_component=100, mav_type=30, camera=None,
>                    loglevel=20)

Create a mavlink Camera server Component, camera argument will normally be a  gstreamer pipeline

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| source_component | int | 100 | used for component indication |
| mav_type | int | 30 | used for heartbeat MAV_TYPE indication |
| camera | NoneType | None | camera  (or FakeCamera for testing) |
| loglevel | int | 20 | logging level |

In [ ]:
# run a server that can receive commands from a client
from UAV.logging import LogLevels

# start a mavlink server that can receive commands from a client
with MAVCom("udpout:localhost:14445", source_system=222, loglevel=LogLevels.DEBUG) as UAV_server:
    # add the camera server components to the server
    UAV_server.add_component(CameraServer(mav_type=mavlink.MAV_TYPE_CAMERA, source_component=mavlink.MAV_COMP_ID_CAMERA, camera=None))
    UAV_server.add_component(CameraServer(mavlink.MAV_COMP_ID_CAMERA2))
    UAV_server.add_component(CameraServer(mavlink.MAV_COMP_ID_CAMERA3))
    
    time.sleep(1)


INFO   | uav.MAVCom      | 37.524 |  mavcom.py:393 | Thread-30 (listen) | MAVLink Mav2: True, source_system: 222
WARNIN | uav.CameraServe | 37.525 | camera_server.py:114 | MainThread         | Component has no camera object
INFO   | uav.CameraServe | 37.526 | component.py:135 | MainThread         | Component Started self.source_component = 100, self.mav_type = 30, self.source_system = 222
WARNIN | uav.CameraServe | 37.527 | camera_server.py:114 | MainThread         | Component has no camera object
INFO   | uav.CameraServe | 37.527 | component.py:135 | MainThread         | Component Started self.source_component = 101, self.mav_type = 30, self.source_system = 222
WARNIN | uav.CameraServe | 37.528 | camera_server.py:114 | MainThread         | Component has no camera object
INFO   | uav.CameraServe | 37.528 | component.py:135 | MainThread         | Component Started self.source_component = 102, self.mav_type = 30, self.source_system = 222


UAV                             
UAV                             
UAV                             


INFO   | uav.CameraServe | 40.529 | component.py:404 | MainThread         | CameraServer closed
INFO   | uav.CameraServe | 40.529 | component.py:404 | MainThread         | CameraServer closed
INFO   | uav.CameraServe | 40.530 | component.py:404 | MainThread         | CameraServer closed
INFO   | uav.MAVCom      | 40.531 |  mavcom.py:442 | MainThread         | MAVCom  closed


In [ ]:
mavlink.enums['MAV_CMD'][527].name

'MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS'

In [ ]:
mavlink.enums['MAV_CMD'][mavlink.MAV_CMD_REQUEST_MESSAGE].name


'MAV_CMD_REQUEST_MESSAGE'

In [ ]:
mavlink.enums['MAV_CMD'][mavlink.MAV_CMD_REQUEST_MESSAGE].name

In [ ]:
mavlink.enums['MAV_CMD'][mavlink.MAV_CMD_REQUEST_MESSAGE]
mavlink.enums['MAV_CMD'].keys()

dict_keys([16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 30, 31, 32, 33, 80, 81, 82, 83, 84, 85, 92, 93, 94, 95, 112, 113, 114, 115, 159, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 220, 221, 222, 223, 224, 240, 241, 242, 243, 245, 246, 252, 260, 300, 400, 401, 410, 500, 510, 511, 512, 519, 520, 521, 522, 525, 526, 527, 528, 529, 530, 531, 532, 600, 601, 1000, 1001, 2000, 2001, 2003, 2004, 2005, 2010, 2500, 2501, 2502, 2503, 2504, 2505, 2510, 2511, 2520, 2600, 2800, 3000, 3001, 4000, 4001, 5000, 5001, 5002, 5003, 5004, 5100, 5200, 10001, 10151, 30001, 30002, 31000, 31001, 31002, 31003, 31004, 31005, 31006, 31007, 31008, 31009, 31010, 31011, 31012, 31013, 31014, 32000, 42000, 42001, 42002, 42003, 42004, 42005, 42006, 42007, 42424, 42425, 42426, 42427, 42428, 42429, 42501, 42502, 42503, 42505, 42600, 42650, 42651, 42700, 42701, 42702, 42703

In [ ]:
CameraServer().mav_cmd_list

[512, 527, 521, 522, 525, 526, 531, 2000, 2001, 2500, 2501, 530, 2502, 2503]

In [ ]:
print("mav_cmd_list")
for cmd in CameraServer().mav_cmd_list:
    cmd_value = mavlink.enums['MAV_CMD'][cmd]
    print(f"{cmd_value.name = } {cmd_value.} {cmd_value.description}")

print("mav_msg_id_list") 
for msg in CameraServer().mav_msg_id_list:
    print(f"{msg = }")

mav_cmd_list
cmd_value.name = 'MAV_CMD_REQUEST_MESSAGE'  Request the target system(s) emit a single instance of a specified message (i.e. a "one-shot" version of MAV_CMD_SET_MESSAGE_INTERVAL).
cmd_value.name = 'MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS'  Request camera capture status (CAMERA_CAPTURE_STATUS)
cmd_value.name = 'MAV_CMD_REQUEST_CAMERA_INFORMATION'  Request camera information (CAMERA_INFORMATION).
cmd_value.name = 'MAV_CMD_REQUEST_CAMERA_SETTINGS'  Request camera settings (CAMERA_SETTINGS).
cmd_value.name = 'MAV_CMD_REQUEST_STORAGE_INFORMATION'  Request storage information (STORAGE_INFORMATION). Use the command's target_component to target a specific component's storage.
cmd_value.name = 'MAV_CMD_STORAGE_FORMAT'  Format a storage medium. Once format is complete, a STORAGE_INFORMATION message is sent. Use the command's target_component to target a specific component's storage.
cmd_value.name = 'MAV_CMD_SET_CAMERA_ZOOM'  Set camera zoom. Camera must respond with a CAMERA_SETTINGS

In [ ]:
# Listed commands are:
CameraServer().list_commands()

Supported Commands: https://mavlink.io/en/messages/common.html#mav_commands
[512, 527, 521, 522, 525, 526, 531, 2000, 2001, 2500, 2501, 530, 2502, 2503]
Supported Message Requests:  https://mavlink.io/en/messages/common.html#messages
[259, 260, 261, 262, 263, 269, 270]


[autoreload of UAV.mavlink.camera_server failed: Traceback (most recent call last):
  File "/home/jn/PycharmProjects/UAV/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/jn/PycharmProjects/UAV/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/usr/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/jn/PycharmProjects/UAV/UAV/mavlink/camera_server.py", line 67, in <module>
    class CameraServer(Component):
  File "/home/jn/PycharmProjects/UAV/UAV/mavlink/camera_server.py", line 107, in CameraServer
    mav_msg_id_list = [ mavlink.enums['MAVLINK_MSG

In [ ]:
doc_class(CameraServer)

---

### CameraServer.close

>      CameraServer.close ()

Close the connection to the camera

---

### Component.count_message

>      Component.count_message (msg)

Count a message by adding it to the message_cnts dictionary. indexed by system and message type

---

### CameraServer.list_commands

>      CameraServer.list_commands ()

List the commands supported by the camera server
https://mavlink.io/en/messages/common.html
https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_INFORMATION
https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_SETTINGS
https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_STORAGE_INFORMATION
https://mavlink.io/en/messages/common.html#MAV_CMD_STORAGE_FORMAT
https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_ZOOM
etc

---

### CameraServer.on_mav_connection

>      CameraServer.on_mav_connection ()

Start the mavlink connection

---

### CameraServer.on_message

>      CameraServer.on_message
>                               (msg:pymavlink.dialects.v20.ardupilotmega.MAVLin
>                               k_command_long_message)

Callback for a command received from the client
This will respond to the mavlink camera and storage focused commands:

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| msg | MAVLink_command_long_message | : mavlink  Message |
| **Returns** | **bool** | **return True to indicate that the message has been handled** |

---

### Component.send_ack

>      Component.send_ack (msg, ack_result:object=0)

Send an ACK message to indicate a command was received.

---

### Component.send_command

>      Component.send_command (target_system:int, target_component:int,
>                              command_id:int, params:list, timeout=0.5)

**Note: async function** 



---

### Component.send_ping

>      Component.send_ping (target_system:int, target_component:int,
>                           ping_num:int=None)

Send self.max_pings * ping messages to test if the server is alive.

---

### Component.set_log

>      Component.set_log (loglevel)

---

### Component.set_mav_connection

>      Component.set_mav_connection (mav_com:MAVCom)

Set the mav_connection for the component

---

### Component.set_source_compenent

>      Component.set_source_compenent ()

Set the source component for the master.mav

---

### Component.set_target

>      Component.set_target (target_system, target_component)

Set the target system and component for the gimbal

---

### Component.wait_ack

>      Component.wait_ack (target_system, target_component, command_id=None,
>                          timeout=0.1)

Wait for an ack from target_system and target_component.

**Note: async function** 



---

### Component.wait_heartbeat

>      Component.wait_heartbeat (remote_mav_type=None, target_system=None,
>                                target_component=None, timeout:int=1)

Wait for a heartbeat from target_system and target_component.

**Note: async function** 





### Camera Client 
> The client is on the ground station PC and is used to send commands to the camera on the companion computer.

In [ ]:
show_doc(CameraClient)

---

### CameraClient

>      CameraClient (source_component, mav_type,
>                    loglevel:UAV.logging.LogLevels=20)

Create a client component to send commands to a companion computer or GCS that will control a camera via a CameraServer instance

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| source_component |  |  | used for component indication |
| mav_type |  |  | used for heartbeat MAV_TYPE indication |
| loglevel | LogLevels | 20 | logging level |

In [ ]:
doc_class(CameraClient)

---

### Component.close

>      Component.close ()

---

### Component.count_message

>      Component.count_message (msg)

Count a message by adding it to the message_cnts dictionary. indexed by system and message type

---

### CameraClient.image_start_capture

>      CameraClient.image_start_capture (target_system=None,
>                                        target_component=None, interval=0,
>                                        count=1)

Start image capture sequence.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| target_system | NoneType | None |  |
| target_component | NoneType | None |  |
| interval | int | 0 | Image capture interval |
| count | int | 1 | Number of images to capture (0 for unlimited) |

---

### CameraClient.image_stop_capture

>      CameraClient.image_stop_capture (target_system=None,
>                                       target_component=None)

Stop image capture sequence

---

### CameraClient.message_callback_cond

>      CameraClient.message_callback_cond (msg_id, target_system,
>                                          target_component, timeout=1)

Register a callback for a message received from the server
Returns the message

**Note: async function** 



---

### CameraClient.on_mav_connection

>      CameraClient.on_mav_connection ()

---

### CameraClient.on_message

>      CameraClient.on_message
>                               (msg:pymavlink.dialects.v20.ardupilotmega.MAVLin
>                               k_message)

Callback for a command received from the server

---

### CameraClient.request_message

>      CameraClient.request_message (msg_id, params=None, target_system=None,
>                                    target_component=None)

Request a message from the camera

**Note: async function** 



---

### Component.send_ack

>      Component.send_ack (msg, ack_result:object=0)

Send an ACK message to indicate a command was received.

---

### Component.send_command

>      Component.send_command (target_system:int, target_component:int,
>                              command_id:int, params:list, timeout=0.5)

**Note: async function** 



---

### CameraClient.send_message

>      CameraClient.send_message (msg)

Send a message to the camera

---

### Component.send_ping

>      Component.send_ping (target_system:int, target_component:int,
>                           ping_num:int=None)

Send self.max_pings * ping messages to test if the server is alive.

---

### CameraClient.set_camera_mode

>      CameraClient.set_camera_mode (target_system=None, target_component=None,
>                                    mode_id=0)

Set the camera mode

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| target_system | NoneType | None |  |
| target_component | NoneType | None |  |
| mode_id | int | 0 | https://mavlink.io/en/messages/common.html#CAMERA_MODE |

---

### CameraClient.set_camera_zoom

>      CameraClient.set_camera_zoom (target_system=None, target_component=None,
>                                    zoom_type=0, zoom_value=1)

Set the camera zoom

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| target_system | NoneType | None |  |
| target_component | NoneType | None |  |
| zoom_type | int | 0 |  |
| zoom_value | int | 1 | 0 to 100 zoom value |

---

### Component.set_log

>      Component.set_log (loglevel)

---

### Component.set_mav_connection

>      Component.set_mav_connection (mav_com:MAVCom)

Set the mav_connection for the component

---

### CameraClient.set_message_callback_cond

>      CameraClient.set_message_callback_cond (msg_id, target_system,
>                                              target_component)

Register a callback condition for a message received from the server

---

### Component.set_source_compenent

>      Component.set_source_compenent ()

Set the source component for the master.mav

---

### Component.set_target

>      Component.set_target (target_system, target_component)

Set the target system and component for the gimbal

---

### CameraClient.storage_format

>      CameraClient.storage_format (target_system=None, target_component=None)

Format storage (for cases where camera has storage)

---

### CameraClient.video_start_capture

>      CameraClient.video_start_capture (target_system=None,
>                                        target_component=None,
>                                        video_stream_id=0, frequency=1)

Start video capture

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| target_system | NoneType | None |  |
| target_component | NoneType | None |  |
| video_stream_id | int | 0 | Video stream id (0 for all streams) |
| frequency | int | 1 | Frequency CAMERA_CAPTURE_STATUS messages should be sent while recording (0 for no messages, otherwise frequency in Hz) |

---

### CameraClient.video_start_streaming

>      CameraClient.video_start_streaming (target_system=None,
>                                          target_component=None,
>                                          video_stream_id=0)

Start video streaming

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| target_system | NoneType | None |  |
| target_component | NoneType | None |  |
| video_stream_id | int | 0 | Video Stream ID (0 for all streams) |

---

### CameraClient.video_stop_capture

>      CameraClient.video_stop_capture (target_system=None,
>                                       target_component=None,
>                                       video_stream_id=0)

Stop video capture

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| target_system | NoneType | None |  |
| target_component | NoneType | None |  |
| video_stream_id | int | 0 | Video stream id (0 for all streams) |

---

### CameraClient.video_stop_streaming

>      CameraClient.video_stop_streaming (target_system=None,
>                                         target_component=None,
>                                         video_stream_id=0)

Stop the video stream

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| target_system | NoneType | None |  |
| target_component | NoneType | None |  |
| video_stream_id | int | 0 | Video Stream ID (0 for all streams) |

---

### Component.wait_ack

>      Component.wait_ack (target_system, target_component, command_id=None,
>                          timeout=0.1)

Wait for an ack from target_system and target_component.

**Note: async function** 



---

### Component.wait_heartbeat

>      Component.wait_heartbeat (remote_mav_type=None, target_system=None,
>                                target_component=None, timeout:int=1)

Wait for a heartbeat from target_system and target_component.

**Note: async function** 



---

### CameraClient.wait_message_callback

>      CameraClient.wait_message_callback (cond, timeout=1)

Wait for the callback for a message received from the server

**Note: async function** 





### Example: Test locally using UDP ports

> on the same machine using UDP ports `14445`  with `server_system_ID=111, client_system_ID=222`


In [ ]:
%autoawait asyncio
import asyncio
async def main():
    MAV_TYPE_GCS = mavutil.mavlink.MAV_TYPE_GCS
    MAV_TYPE_CAMERA = mavutil.mavlink.MAV_TYPE_CAMERA
    
    con1, con2 = "udpin:localhost:14445", "udpout:localhost:14445"
    # con1, con2 = "/dev/ttyACM0", "/dev/ttyUSB0"
    with MAVCom(con1, source_system=111) as client:
        with MAVCom(con2, source_system=222) as server:
            cam:CameraClient = client.add_component(
                CameraClient(mav_type=MAV_TYPE_GCS, source_component=11))
            # server.add_component(CameraServer(mav_type=MAV_TYPE_CAMERA, source_component=22, camera=cam_fake1, debug=False))
            server.add_component(CameraServer(mav_type=MAV_TYPE_CAMERA, source_component=22, camera=None))
    
            await cam.wait_heartbeat(target_system=222, target_component=22, timeout=1)
            time.sleep(0.1)
    
            msg = await cam.request_message(mavlink.MAVLINK_MSG_ID_CAMERA_INFORMATION, target_system=222, target_component=22)

            print( f"MAVLINK_MSG_ID_CAMERA_INFORMATION {msg}")
            
            # msg = await cam.request_storage_information()
            # print (msg)
            
            time.sleep(1)
            
await main()

INFO   | uav.MAVCom      | 54.661 |  mavcom.py:393 | Thread-5 (listen)  | MAVLink Mav2: True, source_system: 111
INFO   | uav.MAVCom      | 54.763 |  mavcom.py:393 | Thread-6 (listen)  | MAVLink Mav2: True, source_system: 222
INFO   | uav.CameraClien | 54.770 | component.py:135 | MainThread         | Component Started self.source_component = 11, self.mav_type = 6, self.source_system = 111
WARNIN | uav.CameraServe | 54.772 | camera_server.py: 92 | MainThread         | Component has no camera object
INFO   | uav.CameraServe | 54.773 | component.py:135 | MainThread         | Component Started self.source_component = 22, self.mav_type = 30, self.source_system = 222


set_mav_connection CameraClient component.py:131 self.mav_com = <MAVCom>
set_mav_connection CameraServer component.py:131 self.mav_com = <MAVCom>
UAV                             
MAVLINK_MSG_ID_CAMERA_INFORMATION CAMERA_INFORMATION {time_boot_ms : 1699, vendor_name : UAV, model_name : FakeCamera, firmware_version : 1, focal_length : 2.799999952316284, sensor_size_h : 3.200000047683716, sensor_size_v : 2.4000000953674316, resolution_h : 640, resolution_v : 480, lens_id : 0, flags : 0, cam_definition_version : 1, cam_definition_uri : , gimbal_device_id : 0}


INFO   | uav.CameraServe | 58.786 | component.py:404 | MainThread         | CameraServer closed
INFO   | uav.MAVCom      | 58.787 |  mavcom.py:442 | MainThread         | MAVCom  closed
INFO   | uav.CameraClien | 00.794 | component.py:404 | MainThread         | CameraClient closed
INFO   | uav.MAVCom      | 00.795 |  mavcom.py:442 | MainThread         | MAVCom  closed


In [ ]:
# | hide
# assert False, "Stop here"

#### Starting a client and server
 > on the same machine using UDP ports `14445`  with `server_system_ID=111, client_system_ID=222`

In [ ]:
#| exports
from UAV.mavlink.mavcom import MAVCom
from UAV.mavlink.component import Component, mavutil
import time

MAV_TYPE_GCS = mavutil.mavlink.MAV_TYPE_GCS
MAV_TYPE_CAMERA = mavutil.mavlink.MAV_TYPE_CAMERA

class Cam1(Component):
    def __init__(self, source_component, mav_type, debug=False):
        super().__init__(source_component=source_component, mav_type=mav_type,
                         debug=debug)

class Cam2(Component):
    def __init__(self, source_component, mav_type, debug=False):
        super().__init__(source_component=source_component, mav_type=mav_type,
                         debug=debug)
class Cli(Component):
    def __init__(self, source_component, mav_type, debug=False):
        super().__init__( source_component=source_component, mav_type=mav_type,
                         debug=debug)

In [ ]:
#| exports
def test_ack():
    # Test sending a command and receiving an ack from client to server
    with MAVCom("udpin:localhost:14445", source_system=111, debug=False) as client:
        with MAVCom("udpout:localhost:14445", source_system=222, debug=False) as server:
            client.add_component(Cli( mav_type=MAV_TYPE_GCS, source_component = 11, debug=False))
            server.add_component(Cam1( mav_type=MAV_TYPE_CAMERA, source_component = 22, debug=False))
            server.add_component(Cam1( mav_type=MAV_TYPE_CAMERA, source_component = 23, debug=False))
            
            for key, comp in client.component.items():
                if comp.wait_heartbeat(target_system=222, target_component=22, timeout=0.1):
                    print ("*** Received heartbeat **** " )
            NUM_TO_SEND = 2
            for i in range(NUM_TO_SEND):
                client.component[11]._test_command(222, 22, 1)
                client.component[11]._test_command(222, 23, 1)
                
            client.component[11]._test_command(222, 24, 1)
    
        print(f"{server.source_system = };  {server.message_cnts = }")
        print(f"{client.source_system = };  {client.message_cnts = }")
        print()
        print(f"{client.source_system = } \n{client.summary()} \n")
        print(f"{server.source_system = } \n{server.summary()} \n")
    
        assert client.component[11].num_cmds_sent == NUM_TO_SEND * 2 + 1
        assert client.component[11].num_acks_rcvd == NUM_TO_SEND * 2
        assert client.component[11].num_acks_drop == 1
        assert server.component[22].num_cmds_rcvd == NUM_TO_SEND
        assert server.component[23].num_cmds_rcvd == NUM_TO_SEND
test_ack()

TypeError: MAVCom.__init__() got an unexpected keyword argument 'debug'

In [ ]:
#| Hide
# assert False, "Stop here"

In [ ]:
show_doc(Component.set_source_compenent)

In [ ]:
show_doc(Component.send_heartbeat)

In [ ]:
show_doc(Component.send_command)

In [ ]:
show_doc(Component.wait_heartbeat)

In [ ]:
show_doc(Component.wait_ack)

In [ ]:
show_doc(Component.send_ping)

#### Test with Serial ports
Test using a Pixhawk connected via telemetry 2 and USB serial ports.
CamClient is set to udpin:localhost:14445 and CamServer is set to udpout:localhost:14435 udpin is so that the client can receive UDP from the mavproxy server at localhost:14445
mavproxy.py --master=/dev/ttyACM1 --baudrate 57600 --out udpout:localhost:14445 mavproxy.py --master=/dev/ttyACM3 --baudrate 57600 --out udpout:localhost:14435

In [ ]:
# Test sending a command and receiving an ack from client to server
with MAVCom("/dev/ttyACM0", source_system=111, debug=False) as client:
    with MAVCom("/dev/ttyUSB0", source_system=222, debug=False) as server:
        client.add_component(Cli(client, mav_type=MAV_TYPE_GCS, source_component = 11, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 22, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 23, debug=False))
        
        for key, comp in client.component.items():
            if comp.wait_heartbeat(target_system=222, target_component=22, timeout=0.1):
                print ("*** Received heartbeat **** " )
        NUM_TO_SEND = 2
        for i in range(NUM_TO_SEND):
            client.component[11]._test_command(222, 22, 1)
            client.component[11]._test_command(222, 23, 1)
            
        client.component[11]._test_command(222, 24, 1)

    print(f"{server.source_system = };  {server.message_cnts = }")
    print(f"{client.source_system = };  {client.message_cnts = }")
    print()
    print(f"{client.source_system = } \n{client.summary()} \n")
    print(f"{server.source_system = } \n{server.summary()} \n")

    assert client.component[11].num_cmds_sent == NUM_TO_SEND * 2 + 1
    assert client.component[11].num_acks_rcvd == NUM_TO_SEND * 2
    assert client.component[11].num_acks_drop == 1
    assert server.component[22].num_cmds_rcvd == NUM_TO_SEND
    assert server.component[23].num_cmds_rcvd == NUM_TO_SEND

> For debugging help see http://localhost:3000/tutorials/mavlink_doc&debug.html and http://localhost:3000/tutorials/mavlink_doc&debug.html#debugging

In [ ]:
#| hide
# from nbdev import nbdev_export
# nbdev.nbdev_export()